# Notebook for training the LSTM Transformer model

In [86]:
from google_drive_fetcher import GoogleDriveFetcher 
import numpy as np
import pandas as pd
from torch import tensor
import torch
from collections import defaultdict
from lstm_transformer.lstm_transformer_classifier import LSTMTransformerClassifier
from streamer import BatchStream 



Configs

In [87]:
FOLDER_URL = "https://drive.google.com/drive/folders/1oWDXaQPOcnVF8I_bFHrVIYTspjE-eBD_?hl=da"

TRAIN_URL = "https://drive.google.com/drive/folders/1Jr95Jgu4uqEKNrYAILOu2cLVHeEKsW7f?hl=da"

TEST_URL = ""

hyperparams = {
    "hidden_size": 64, 
    "num_lstm_layers": 2, 
    "num_heads": 8, 
    "num_transformer_layers": 2, 
    "batch_size" : 32, 
    "batch_first": True
}

In [88]:
class BatchStream2: 
    def __init__(self):
        self.train_url = TRAIN_URL
        self.gdf = GoogleDriveFetcher(URL = TRAIN_URL)
        self.items = self.gdf.list_directory(self.train_url)   

    def stream_next(self) -> tuple[list[torch.Tensor], list[str]]:
        df = self.stream_next_df() 
        batch_X = [self.convert_array_to_tensor(np.array(arr)) for arr in df.iloc[:, 0].values]
        batch_Y = [port for port in df["Port"].values]
        return batch_X, batch_Y


    def stream_next_df(self) -> pd.DataFrame:
        item = self.get_next_item()
        df = self.item_to_df(item)
        return df

    def item_to_df(self, item: dict) -> pd.DataFrame: 
        url = "https://drive.google.com/drive/folders/" + item['id'] + "?hl=da"
        df = self.gdf.fetch_parquet_to_dataframe(url)
        return df 

    
    def get_next_item(self) -> dict:
        return self.items.pop(0)

    def convert_array_to_tensor(self, array: np.ndarray) -> torch.Tensor:
        t = torch.tensor(np.stack(array), dtype=torch.float32)
        return t

    

    def has_items_left(self) -> bool:
        return len(self.items) > 0  


## Finding Meta Data About The Ports

In [89]:
batchstreamer = BatchStream() 
ports = defaultdict(int) 

print("Streaming Ports...")

#while batchstreamer.has_items_left(): 
#    
#    df = batchstreamer.stream_next_df() 
#    unique_ports_in_df = df['Port'].unique() 
#    for port in unique_ports_in_df: 
#        ports[port] += 1 
#    ports_df = pd.DataFrame(list(ports.items()), columns=['Port', 'Count']) 
#    ports_df.to_csv("port_counts.csv", index=False)

Streaming Ports...


## Training LSTM-TRANSFORMER

In [ ]:
ports_df = pd.read_csv("port_counts.csv")  
port_encoder = {port_name: i for i, port_name in enumerate(ports_df["Port"])} 

hyperparams["output_size"] = len(port_encoder) 
hyperparams["input_size"] = len(batchstreamer.item_to_df(batchstreamer.items[0]).columns) 



classifier = LSTMTransformerClassifier(
    input_size=hyperparams["input_size"],
    hidden_size=hyperparams["hidden_size"],
    num_layers=hyperparams["num_lstm_layers"],
    num_heads=hyperparams["num_heads"],
    num_transformer_layers=hyperparams["num_transformer_layers"],
    output_size=hyperparams["output_size"],
    batch_first=hyperparams["batch_first"]
    
)

In [91]:

while batchstreamer.has_items_left(): 
    batch_X, batch_Y = batchstreamer.stream_next() 
    batch_Y = [port_encoder[port] for port in batch_Y]

    
    # Process batch_X and batch_Y as needed

KeyboardInterrupt: 

In [ ]:
gdf_train = GoogleDriveFetcher(URL = TRAIN_URL) 

files_dict = gdf_train.list_directory(TRAIN_URL)


keep_going = True 

while keep_going: 
    item = dict(files_dict.pop(0)) #remote a item element 
    
    print("Downloading file...")
    df = item_to_df(item = item, gdf = gdf_train)
    if df is None: 
        raise ValueError("Downlad did not complete")
    print("Downloaded file!!")


    print("Extracting Features & converting to Tensor...")
    batch_X = [convert_array_to_tensor(np.array(arr)) for arr in df.iloc[:, 0].values] 
    batch_Y = []
    print("Converted to tensors!")



    keep_going = False




for item in file_dict[:2]: 
    url = "https://drive.google.com/drive/folders/" + item['id'] + "?hl=da"
    print(f"Downloaded folder: {item['name']}")

    print("Downlaod file")
    df = gdf_train.fetch_parquet_to_dataframe(url)

    files.append(df)


Downloaded file!!
Extracting Features & converting to Tensor...
Downloaded folder: 0be7601f46334f1da4d3677131bafe94-0.parquet Binary
Downlaod file
Downloaded folder: 2e7c7667a03140a09385d8f4974930d1-0.parquet Binary
Downlaod file


In [ ]:
batch_X

[tensor([[0.0000e+00, 5.6889e+01, 1.2478e+01, 0.0000e+00, 1.4280e+02],
         [1.8000e+02, 5.6889e+01, 1.2478e+01, 0.0000e+00, 2.1950e+02],
         [2.9200e+02, 5.6889e+01, 1.2478e+01, 0.0000e+00, 2.1950e+02],
         ...,
         [2.7910e+04, 5.6303e+01, 1.1382e+01, 4.2184e+00, 2.0680e+02],
         [2.7930e+04, 5.6302e+01, 1.1382e+01, 4.0641e+00, 2.0640e+02],
         [2.7941e+04, 5.6302e+01, 1.1381e+01, 4.1156e+00, 2.0620e+02]]),
 tensor([[0.0000e+00, 5.6252e+01, 7.2163e+00, 3.0867e+00, 2.0370e+02],
         [1.5000e+01, 5.6251e+01, 7.2158e+00, 2.9323e+00, 2.0060e+02],
         [2.5000e+01, 5.6251e+01, 7.2156e+00, 2.9323e+00, 1.9600e+02],
         ...,
         [1.6835e+04, 5.5810e+01, 6.7582e+00, 3.7554e+00, 2.0390e+02],
         [1.6846e+04, 5.5809e+01, 6.7579e+00, 3.7554e+00, 2.0440e+02],
         [1.6895e+04, 5.5808e+01, 6.7563e+00, 3.8583e+00, 2.1760e+02]]),
 tensor([[0.0000e+00, 5.4953e+01, 1.3823e+01, 5.1444e+00, 7.2300e+01],
         [2.6000e+01, 5.4954e+01, 1.3825e+01,

In [ ]:
import numpy as np
t = torch.tensor(np.stack(files[0].iloc[0, 0]), dtype=torch.float32)

t

tensor([[0.0000e+00, 5.6889e+01, 1.2478e+01, 0.0000e+00, 1.4280e+02],
        [1.8000e+02, 5.6889e+01, 1.2478e+01, 0.0000e+00, 2.1950e+02],
        [2.9200e+02, 5.6889e+01, 1.2478e+01, 0.0000e+00, 2.1950e+02],
        ...,
        [2.7910e+04, 5.6303e+01, 1.1382e+01, 4.2184e+00, 2.0680e+02],
        [2.7930e+04, 5.6302e+01, 1.1382e+01, 4.0641e+00, 2.0640e+02],
        [2.7941e+04, 5.6302e+01, 1.1381e+01, 4.1156e+00, 2.0620e+02]])

In [ ]:
files[0]

,input_segment,target_segment,Port,SampleID
0,"[[0.0, 56.888966, 12.478322, 0.0, 142.8], [180...","[[27960.0, 56.301434, 11.38072, 4.1669965, 206...",no_port,209336000_0_0
1,"[[0.0, 56.252, 7.21629, 3.086664, 203.7], [15....","[[16969.0, 55.8054, 6.7542, 3.85833, 204.9], [...",no_port,209341000_0_0
2,"[[0.0, 54.9532, 13.8231, 5.14444, 72.3], [26.0...","[[21277.0, 55.5572, 15.2086, 5.195884, 60.7], ...",no_port,209664000_0_0
3,"[[0.0, 57.6901, 10.6425, 0.0, 272.7], [78.0, 5...","[[48241.0, 57.6793, 10.6204, 0.0, 272.4], [482...",no_port,210051000_1_0
4,"[[0.0, 54.230137, 11.978773, 0.0514444, 33.8],...","[[15278.0, 54.1532, 12.123695, 0.7202216, 164....",DERSK,210261000_0_0
...,...,...,...,...
388,"[[0.0, 56.0104, 12.694, 4.4756627, 5.3], [10.0...","[[60027.0, 56.0299, 12.6918, 0.0, 8.1], [60181...",SEHEL,636017570_1_3
389,"[[0.0, 56.0104, 12.694, 4.4756627, 5.3], [10.0...","[[58947.0, 56.0299, 12.6918, 0.0, 8.1], [59101...",SEHEL,636017570_1_4
390,"[[0.0, 55.0555, 13.2777, 4.629996, 313.9], [17...","[[27720.0, 56.0054, 12.6839, 4.6814404, 344.7]...",no_port,636018261_0_0
391,"[[0.0, 57.5495, 11.6574, 0.1543332, 49.4], [18...","[[81952.0, 57.5485, 11.6475, 0.0, 82.5], [8205...",no_port,636019842_0_0
